<div class="alert alert-block alert-info">
  <center style="font-size: 2em;">Case 4 - Fidelidade</center>
  <center style="font-size: 1.5em;">Group T</center>
  <center style="font-size: 1.2em;">Spring Semester/2025</center>
</div>

 #### This Project was made by:

André Lourenço <br>
20240743 <br>
20240743@novaims.unl.pt <br>

Emir Kamiloglu <br>
20240945 <br>
20240945@novaims.unl.pt <br>

Manuel Andrade <br>
20240571 <br>
20240571@novaims.unl.pt <br>

Victor Silva <br>
20240663 <br>
20240663@novaims.unl.pt  <br>

Rute D’Alva Teixeira <br>
20240667 <br>
20240667@novaims.unl.pt <br>

# Table of Contents

**1. [Environment Setup](#setup)**
- 1.1 [Required Installs](#pip)
- 1.2 [Importing Libraries](#libraries)
- 1.3 [Initializing Azure OpenAI Client](#client)


**2. [Text Pre-Processing](#preprocess)**
- 2.1 [PDF Text and Image Extraction](#extraction)
- 2.2 [Loading Extracted Text](#loading)
- 2.3 [Text Chunking](#chunking)


**3. [RAG - Retrieval Augmentation System](#RAG)**
- 3.1 [Getting Text Embeddings](#embeddings)  
- 3.2 [Building FAISS Index](#faiss)  
- 3.3 [Searching the Index](#index)  

**4. [Chatbot Setup](#chatbot-setup)**
- 4.1 [Token-Aware Chunk Truncation](#truncation)  
- 4.2 [Defining and Querying the Chatbot](#chatbot)  

**5. [Performance evaluation](#evaluation)**  
- 5.1 [Defining Evaluation Criteria](#criteria)  
- 5.2 [Defining Test Dictionaries](#testing)  
- 5.3 [Evaluation Results](#results)

**6. [Exporting variables for interface](#export)**  



<a class="anchor" id="setup"></a>

# 1. Environment Setup

<a class="anchor" id="pip"></a>

## 1.1 Required Installs

In [6]:

!pip install pdfplumber pymupdf faiss-cpu openai tiktoken

!pip install python-dotenv

!pip install --upgrade openai

!pip install openai numpy


<a class="anchor" id="libraries"></a>

## 1.2 Importing Libraries

In [5]:
# -------------------------------
# Core Libraries
import pandas as pd
import numpy as np
import os


#----------------------------------
# Text Preprocessing/ Extraction
import pdfplumber

#---------------------------------
#Setup Environment
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
#------------------------------------
#RAG
import faiss
import openai
import tiktoken


#-------------------
#COISINE
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\20240667\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\20240667\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\20240667\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    s

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

<a class="anchor" id="client"></a>


## 1.3. Initializing Azure OpenAI Client

To access OpenAi's API key, we made use of NOVA IMS' resources, following these steps:

1) Acquiring API key at faculty's provided website
2) Defining API key on a text file, assigning it to variable `("AZURE_OPENAI_KEY")`
3) Saving file as .env file on the same directory as Python script


Then, we loaded the .env file, and initialized the client as defined below:

In [ ]:
# Load env variables
load_dotenv()

True

In [ ]:
# -----------Initializing the AzureOpenAI client
client = AzureOpenAI(
    azure_endpoint="https://ai-bcds.openai.azure.com/",  # Provided by NOVA IMS
    api_key=os.getenv("AZURE_OPENAI_KEY"),               # Securely loaded from .env
    api_version="2025-01-01-preview"                     # Version provided or current
)

<a class="anchor" id="preprocess"></a>

# 2. Text Preprocessing

<a class="anchor" id="extraction"></a>

## 2.1. PDF Text and Image Extraction

Uncomment and run if you are running for the first time, or if you update the source folder's content.

After running this piece of code for the first time, you can Comment it, since the result of this extraction is already stored in "outputs" folder created in directory.

Note: To ensure this cell runs correctly, the folder on which you store the subfolders with docs to upload should be named "pdf_chatbot". Alternatively, you can edit the function 'def process_all_pdfs()' and update 'root_input_folder' parameter with the correct folder name in your directory.

In [ ]:
import os
from pathlib import Path
import pdfplumber
import fitz  # PyMuPDF

#--------------------------------------------------------------------------
# This code will recursively look for pdfs in every subfolder it finds and extract text, images as outputs onto a new folder

def find_all_pdfs(root_folder):
    pdf_files = []
    for dirpath, _, filenames in os.walk(root_folder):
        for file in filenames:
            if file.lower().endswith(".pdf"):
                pdf_files.append(os.path.join(dirpath, file))
    return pdf_files

def extract_text_and_tables(pdf_path):
    # <-- YOU NEED TO IMPLEMENT THIS function to extract text + tables using pdfplumber or fitz
    # For example (basic text extraction):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n\n"
    return text

def extract_images(pdf_path, output_dir):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    doc = fitz.open(pdf_path)

    for page_number in range(len(doc)):
        page = doc[page_number]
        image_list = page.get_images(full=True)

        for img_index, img in enumerate(image_list, 1):
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)

            # If it's CMYK or other, convert to RGB first
            if pix.n > 4:
                pix = fitz.Pixmap(fitz.csRGB, pix)

            # Save PNG
            image_path = f"{output_dir}/page{page_number+1}_img{img_index}.png"
            try:
                pix.save(image_path)
            except ValueError:
                # If saving PNG still fails, fallback to save as JPG
                image_path_jpg = image_path.replace(".png", ".jpg")
                pix.save(image_path_jpg)
                print(f"Saved as JPG instead: {image_path_jpg}")

            pix = None



def process_pdf(pdf_path, root_input_folder, root_output_folder):
    # Calculate relative path to preserve folder structure in outputs
    relative_path = os.path.relpath(pdf_path, root_input_folder)
    relative_folder = os.path.dirname(relative_path)

    # Prepare output folders
    output_text_folder = os.path.join(root_output_folder, relative_folder)
    output_images_folder = os.path.join(output_text_folder, "images")
    os.makedirs(output_text_folder, exist_ok=True)
    os.makedirs(output_images_folder, exist_ok=True)

    # Extract text + tables
    extracted_text = extract_text_and_tables(pdf_path)

    # Save text file next to images folder
    text_file_path = os.path.join(output_text_folder, "extracted_text.txt")
    with open(text_file_path, "w", encoding="utf-8") as f:
        f.write(extracted_text)

    # Extract images
    extract_images(pdf_path, output_images_folder)

    print(f"Processed: {pdf_path}")
    print(f"  Text saved to: {text_file_path}")
    print(f"  Images saved to: {output_images_folder}\n")

def process_all_pdfs(root_input_folder="pdf_chatbot", root_output_folder="outputs"): #-------> "pdf_chatbot" or the name of the folder where this script and the docs are
    pdf_files = find_all_pdfs(root_input_folder)
    print(f"Found {len(pdf_files)} PDFs to process.\n")
    for pdf_file in pdf_files:
        process_pdf(pdf_file, root_input_folder, root_output_folder)

if __name__ == "__main__":
    process_all_pdfs()


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Found 20 PDFs to process.



CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Processed: pdf_chatbot\Fidelidade_Business Case_NOVA IMS.pdf
  Text saved to: outputs\extracted_text.txt
  Images saved to: outputs\images



CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


...

Saved as JPG instead: outputs\Savings - Documents\My Savings - Public Information\images/page1_img1.jpg
Processed: pdf_chatbot\Savings - Documents\My Savings - Public Information\IPC+DIF_Fid_Savings_Dinâmico (ICAE Ações)-Fidelidade Savings_jan2025.pdf
  Text saved to: outputs\Savings - Documents\My Savings - Public Information\extracted_text.txt
  Images saved to: outputs\Savings - Documents\My Savings - Public Information\images



CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Saved as JPG instead: outputs\Savings - Documents\My Savings - Public Information\images/page1_img1.jpg
Processed: pdf_chatbot\Savings - Documents\My Savings - Public Information\IPC+DIF_Fid_Savings_ESG (ICAE Ações)-Fidelidade Savings_jan2025.pdf
  Text saved to: outputs\Savings - Documents\My Savings - Public Information\extracted_text.txt
  Images saved to: outputs\Savings - Documents\My Savings - Public Information\images



CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Saved as JPG instead: outputs\Savings - Documents\My Savings - Public Information\images/page1_img1.jpg
Processed: pdf_chatbot\Savings - Documents\My Savings - Public Information\IPC+DIF_Fid_Savings_Proteção-Fidelidade Savings_jan2025.pdf
  Text saved to: outputs\Savings - Documents\My Savings - Public Information\extracted_text.txt
  Images saved to: outputs\Savings - Documents\My Savings - Public Information\images



To access OpenAi's API key, we made use of NOVA IMS' resources, following these steps:

1) Acquiring API key at faculty's provided website
2) Defining API key on a text file, assigning it to variable `("AZURE_OPENAI_KEY")`
3) Saving file as .env file on the same directory as Python script


Then, we loaded the .env file, and initialized the client as defined below:

In [ ]:
# Load env variables
load_dotenv()

True

In [ ]:
# -----------Initializing the AzureOpenAI client
client = AzureOpenAI(
    azure_endpoint="https://ai-bcds.openai.azure.com/",  # Provided by NOVA IMS
    api_key=os.getenv("AZURE_OPENAI_KEY"),               # Securely loaded from .env
    api_version="2025-01-01-preview"                     # Version provided or current
)

<a class="anchor" id="loading"></a>

## 2.2. Loading Extracted Text

Loading all .txt files under the outputs folder into memory.

In [ ]:
def load_all_texts(root_text_folder="outputs"):
    all_texts = []
    for dirpath, _, files in os.walk(root_text_folder):
        for file in files:
            if file.endswith(".txt"):
                filepath = os.path.join(dirpath, file)
                with open(filepath, "r", encoding="utf-8") as f:
                    all_texts.append(f.read())
    return all_texts

all_texts = load_all_texts()


<a class="anchor" id="chunking"></a>

## 2.3. Text Chunking

Splitting text into chunks by paragraph

In [ ]:
def split_text_into_chunks(text, max_chunk_size=700):
    import textwrap
    # Simple split by paragraphs and further split if too long
    paragraphs = text.split("\n\n")
    chunks = []
    for para in paragraphs:
        if len(para) <= max_chunk_size:
            chunks.append(para.strip())
        else:
            # split long paragraphs into smaller chunks by max_chunk_size chars
            sub_chunks = textwrap.wrap(para, max_chunk_size)
            chunks.extend([sc.strip() for sc in sub_chunks])
    return chunks

chunks = []
for text in all_texts:
    chunks.extend(split_text_into_chunks(text))


In [ ]:
print("Number of chunks:", len(chunks))

Number of chunks: 353


<a class="anchor" id="RAG"></a>

# 3. RAG - Retrieval Augmentation System

<a class="anchor" id="embeddings"></a>

## 3.1. Getting Text embeddings

In [ ]:
def get_embedding(text):
    try:
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=text
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Embedding failed for input: {text[:30]}... | Error: {e}")
        return None


# Normalize embeddings for cosine similarity
def normalize_embeddings(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

# Example usage
embedding = get_embedding("Hello world")
print(embedding)

[-0.005540425889194012, 0.0047363233752548695, -0.015009919181466103, -0.027093535289168358, -0.015173893421888351, 0.015173893421888351, -0.0176082756370306, 0.009554633870720863, -0.00942219328135252, -0.030801868066191673, 0.02631150558590889, 0.011169145815074444, -0.023397814482450485, -0.009510486386716366, 0.007700467016547918, 0.010450183413922787, 0.027572842314839363, -0.012581843882799149, 0.012783657759428024, 0.014845944941043854, -0.007164398208260536, -0.003342545125633478, 0.0026251592207700014, 0.007183318492025137, -0.019777776673436165, -0.003979520406574011, 0.010633077472448349, -0.017456915229558945, 0.0280773788690567, -0.030928000807762146, 0.003411918645724654, -0.006385522428900003, -0.007643706630915403, -0.019626416265964508, 0.00947895273566246, -0.01697760634124279, 0.002305094851180911, -0.013332339935004711, 0.020067883655428886, -0.017847929149866104, 0.007240078877657652, 0.009636620059609413, 0.012178216129541397, -0.022590558975934982, -0.03299659490

In [ ]:
chunk_embeddings = []
for i, chunk in enumerate(chunks):
    emb = get_embedding(chunk)
    if emb is not None:
        chunk_embeddings.append(np.array(emb).astype('float32'))
    else:
        print(f"[WARNING] No embedding returned for chunk {i}")

if not chunk_embeddings:
    raise ValueError("❌ No embeddings were created. Check your API key, model name, or rate limits.")


<a class="anchor" id="faiss"></a>

## 3.2.  Building FAISS Index

In [ ]:
# Build FAISS index with cosine similarity
def build_faiss_index_cosine(chunks):
    chunk_embeddings = []
    for chunk in chunks:
        emb = get_embedding(chunk)  # Your existing embedding function
        if emb is not None:
            chunk_embeddings.append(np.array(emb).astype('float32'))
    embedding_matrix = np.vstack(chunk_embeddings)
    embedding_matrix = normalize_embeddings(embedding_matrix)

    dim = embedding_matrix.shape[1]
    index = faiss.IndexFlatIP(dim)  # Inner Product for cosine similarity
    index.add(embedding_matrix)
    return index, chunk_embeddings


<a class="anchor" id="index"></a>

## 3.3. Searching the Index

In [ ]:
# Search with cosine similarity
def search_index_cosine(query, index, chunks, k=5):
    query_embedding = get_embedding(query)
    if query_embedding is None:
        print("Failed to get embedding for query.")
        return []
    query_vector = np.array(query_embedding).astype('float32').reshape(1, -1)
    query_vector /= np.linalg.norm(query_vector)

    distances, indices = index.search(query_vector, k)
    results = [chunks[i] for i in indices[0] if i < len(chunks)]
    similarities = distances[0]  # Cosine similarity scores
    return list(zip(results, similarities))

# Example usage:
index, chunk_embeddings = build_faiss_index_cosine(chunks)
results = search_index_cosine("your query text here", index, chunks, k=5)

for res, score in results:
    print(f"Score: {score:.4f}, Text: {res}")

Score: 0.8076, Text: Context
Score: 0.8068, Text: What is your task?
Score: 0.7799, Text: YOU WILL NEED TO TRAIN THE CHATBOT WITH RELEVANT
INFORMATION ABOUT OUR PRODUCTS
Here are the key documents you should upload to the chatbot’s system:
1. Product Information & Comparisons (“My Savings” & “PPR Evoluir”).*
2. Insurance Agent FAQs (common concerns when advising clients).*
3. Main Competitors’ Products (comparative analysis with our products).
4. Other Relevant Information (Tax, Regulatory Guidelines, Financial Literacy).
This allows the chatbot to answer questions, extract information,
and provide assistance based on the content of these documents.
*Furtherdocumentswillbesharedbyemail.
Score: 0.7764, Text: WE CHALLENGE YOUR TEAM TO DEVELOP AN AI-POWERED ASSISTANT TO SUPPORT OUR INSURANCE AGENTS Your chatbot should meet these 3 key goals: Assist in Client Q&A Provide Useful Information Support users with common questions, especially about: Ensure the chatbot delivers instant and accura

<a class="anchor" id="chatbot-setup"></a>

# 4. Chatbot  Setup

<a class="anchor" id="truncation"></a>

## 4.1. Token-Aware Chunk Truncation

Slicing chunks by tokens instead of characters, to ensure the integrity of the sentences:

In [15]:
def truncate_tokens(text, max_tokens=300):
    encoding = tiktoken.encoding_for_model("gpt-4o-mini")
    tokens = encoding.encode(text)
    return encoding.decode(tokens[:max_tokens])

<a class="anchor" id="chatbot"></a>

## 4.2. Defining and querying the chatbot


In [16]:
# --- Function to call chatbot ---
def get_chatbot_answer(question, index=index, chunks=chunks, k=5):
    retrieved_context = search_index_cosine(question, index, chunks, k=k)
    max_tokens = 300  # adjust as needed
    context = "\n\n".join([chunk for chunk, score in retrieved_context])
    context = truncate_tokens(context, max_tokens=300)

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a sales assistant supporting agents during client interactions. "
                                              "Your responses are based on Fidelidade's official documentation, the context provided and certified, accurate sources."
                                              "Your language is clear, polished, and client-friendly. "
                                              "If you don't know the answer to a question related to Fidelidade, respond with: "
                                              "'Thank you for this question. I will cross-check this internally and get back to you to ensure I provide accurate information.' "
                                              "Your scope is to provide information about Fidelidade's insurance products, their competitors products (comparison) and financial literacy. When answering questions about Financial literacy that are not in the documents, you will cite the source and only use credible portuguese, preferably governamental sources or from public institutions."
                                              "If a question is outside your scope, respond with: "
                                              "'This question is outside of my reach. I would love to help you with questions related to Fidelidade products, their competitors products (comparison) or financial literacy. Can I assist you within those boundaries?'"
                                              "Answer in the language you were prompted in: if in portuguese, answer with portuguese from Portugal, not Brazil"
                },
                {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}"},
            ],
            temperature=0.3,  # PREVIOUSLY, IT WASNT EXPLICIT ON THE FUNCTION SO IT WAS AT 1 BY DEFAULT
            seed=42,    
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Chatbot failed to answer question: {question[:30]}... | Error: {e}")
        return ""
    


In [17]:
get_chatbot_answer("O Fidelidade Savings permite dedução no IRS?")

# do not define "question" input 

'O Fidelidade Savings, enquanto contrato de seguro de vida, não permite dedução no IRS. As contribuições para seguros de vida não são dedutíveis para efeitos de IRS, ao contrário de outros produtos de poupança ou investimento que possam ter benefícios fiscais. No entanto, é importante considerar que os rendimentos obtidos são tributados em IRS conforme as taxas efetivas mencionadas, dependendo do ano de resgate ou vencimento.\n\nSe precisar de mais informações sobre este produto ou outros aspectos relacionados, estou à disposição para ajudar!'

<a class="anchor" id="evaluation"></a>


# 5. Performance evaluation

<a class="anchor" id="criteria"></a>

## 5.1. Evaluation Criteria

To evaluate our chabot's performance, we introduce the `Cosine Similarity` metric. This metric evaluates the cosine of the angle between two vectors, with results ranging from -1 to 1:

- `closer to -1`: very dissimilar
- `closer to  1`: very similar

Function is the defined below:

In [18]:
# --- Evaluation function ---
def evaluate_similarity(test_dict, label=""):
    results = []

    print(f"\n📂 Starting evaluation for: {label}")
    for question, expected_answer in test_dict.items():
        print(f"\n🔍 Testing: {question}")

        chatbot_answer = get_chatbot_answer(question)
        print(f"🤖 Chatbot answer: {chatbot_answer[:100]}...")

        expected_embedding = get_embedding(expected_answer)
        chatbot_embedding = get_embedding(chatbot_answer)

        if expected_embedding and chatbot_embedding:
            similarity = cosine_similarity(
                [expected_embedding],
                [chatbot_embedding]
            )[0][0]
            print(f"✅ Cosine similarity: {similarity:.4f}")
            results.append((question, similarity))
        else:
            print("⚠️ Skipping due to missing embeddings.")
            results.append((question, None))

    return results

<a class="anchor" id="testing"></a>

## 5.2. Defining test dictionaries

In [19]:
# --- Test dictionaries ---
fidelidade_mysavings_test_dict = {
    "O Fidelidade Savings é mais indicado para que perfil de risco?": """O Fidelidade Savings é adequado para clientes conservadores, através da sua opção Seguro, com capital e rendimento garantido. Para clientes que gostam moderadamente do risco, através da opção Proteção com garantia de 90% do capital investido no vencimento e exposição ao Índice Multiativos. Para clientes propensos ao risco através da opção Dinâmico com 100% de exposição ao Índice Multiativos. Também está disponível a opção ESG, que investe no índice Multiativos ESG, com risco mas ESG, para clientes com preocupações Ambientais, Sociais e de Governança.""",
    "O Fidelidade Savings permite dedução no IRS?": """Não. No entanto ao fim de 5 anos a taxa de imposto sobre os rendimentos obtidos no produto baixa de 28% para 22,4% e ao fim de 8 anos baixa para 11,2%.""",
    "Tenho de pagar algum imposto sobre os lucros, ou o rendimento já é líquido?": """Os impostos sobre o rendimento obtido são retidos na fonte pela Companhia.""",
    "Qual é a comissão de gestão anual do Fidelidade Savings?": """Nas opções com risco a comissão de Gestão é no máximo 1,2%. Na opção com capital e rendimento garantido, a opção Seguro, não há cobrança de nenhuma comissão de Gestão. Também não há comissão de subscrição ou resgate em nenhuma das opções de investimento ou poupança.""",
    "Quais os riscos associados ao Fidelidade Savings para o cliente?": """Depende da opção escolhida. A opção Seguro tem capital e rendimento garantido. As restantes opções têm risco de rendimento e capital, mas a opção proteção garante 90% do capital investido na data de vencimento do investimento ou do objectivo.""",
    "Posso subscrever e resgatar em qualquer momento?": """Sim posso. Ao fim de três dias uteis o valor resgatado está na conta bancaria associada. Não há encargos de resgate."""
}

fidelidade_pprevoluir_test_dict = {
    "Que tipo de investimento é o PPR Evoluir — Capital e rendimento garantido ou tem risco?": """Ambos. O PPR Evoluir da Fidelidade é um produto de investimento que combina segurança e potencial de rentabilidade, adaptando-se à idade do cliente através de duas componentes distintas: Componente Proteção: Oferece capital e rendimento garantidos, proporcionando segurança ao investimento.; Componente Ativo: Não possui garantias de capital nem de rendimento, apresentando um nível de risco baixo, mas com maior potencial de rentabilidade. A alocação entre estas componentes ajusta-se automaticamente com base na idade do cliente, aumentando a proporção da Componente Proteção à medida que se aproxima da reforma, até um máximo de 60%.""",
    "Posso escolher a componente que pretendo(Ativo ou Proteção)?": """Não. O contrato apenas admite a recomposição automática, pelo que o Tomador do Seguro não tem direito a alterar a composição do investimento.""",
    "Existe diferença fiscal entre fazer resgate/reembolso antes ou depois dos 5 anos?": """Sim. Fora das condições legais - resgatar o PPR Evoluir antes dos 5 anos implica uma tributação mais elevada (21,5%). Após os 5 anos, a tributação diminui (Entre 5 e 8 anos: 17,2%; Mais de 8 anos: 8,6%, sendo necessário que 35% das entregas tenham sido feitas nos primeiros 5 anos para beneficiar da taxa reduzida. Dentro das condições legais (resgate a qualquer tempo) – É possível resgatar antes dos 5 anos sem penalizações (8%) se for por um destes motivos legais: o Desemprego de longa duração o Incapacidade permanente o Doença grave sua ou de familiar""",
    "Se tiver feito uma entrega de 1.000€ há 10 anos e outra de 100.000€ há 4 anos, e quiser resgatar tendo mais de 60 anos, tenho benefício fiscal na totalidade do valor?": """Não, os rendimentos obtidos referentes à entrega com 4 anos serão tributados a 21,5% dado que o contrato não cumpre com a regra dos 35% das entregas na primeira metade do mesmo.""",
    "Qual é a comissão de gestão anual do PPR Evoluir?": """Depende da componente de Investimento do PPR Evoluir. Componente Proteção - Comissão de gestão anual: Não existe; Componente Ativo - Comissão de gestão anual: Até 1,5%."""
}

<a class="anchor" id="results"></a>

## 5.3. Evaluation Results

In [20]:
## temperature = 0.3


# --- Run evaluation for both dictionaries ---
results_savings = evaluate_similarity(fidelidade_mysavings_test_dict, label="Fidelidade MySavings")
results_ppr = evaluate_similarity(fidelidade_pprevoluir_test_dict, label="PPR Evoluir")

# --- Summary ---
print("\n📊 Evaluation Summary: Fidelidade MySavings")
for q, sim in results_savings:
    print(f"- {q[:40]}... → Similarity: {sim:.4f}" if sim else f"- {q[:40]}... → Skipped")

print("\n📊 Evaluation Summary: PPR Evoluir")
for q, sim in results_ppr:
    print(f"- {q[:40]}... → Similarity: {sim:.4f}" if sim else f"- {q[:40]}... → Skipped")


📂 Starting evaluation for: Fidelidade MySavings

🔍 Testing: O Fidelidade Savings é mais indicado para que perfil de risco?
🤖 Chatbot answer: O Fidelidade Savings é mais indicado para investidores com um perfil de risco moderado a elevado. Es...
✅ Cosine similarity: 0.9532

🔍 Testing: O Fidelidade Savings permite dedução no IRS?
🤖 Chatbot answer: O Fidelidade Savings, enquanto contrato de seguro de vida, não permite dedução no IRS. As contribuiç...
✅ Cosine similarity: 0.8282

🔍 Testing: Tenho de pagar algum imposto sobre os lucros, ou o rendimento já é líquido?
🤖 Chatbot answer: Os rendimentos provenientes de produtos de PPR (Planos de Poupança Reforma) em Portugal são tributad...
✅ Cosine similarity: 0.8537

🔍 Testing: Qual é a comissão de gestão anual do Fidelidade Savings?
🤖 Chatbot answer: A comissão de gestão anual do Fidelidade Savings varia consoante a opção escolhida. Para as Opções P...
✅ Cosine similarity: 0.9183

🔍 Testing: Quais os riscos associados ao Fidelidade Savings p

<a class="anchor" id="export"></a>

# 6. Exporting variables for interface

In [21]:
import pickle

with open("chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)


In [22]:
import faiss

faiss.write_index(index, "faiss_index.index")
